### Required IAM Policy
To use the inference profile, attach the following IAM policy to your role or user:

```
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": [
                "bedrock:InvokeModel",
                "bedrock:InvokeModelWithResponseStream"
            ],
            "Resource": [
                "arn:aws:bedrock:us-west-2:794038231401:inference-profile/us.amazon.nova-pro-v1:0"
            ]
        },
        {
            "Effect": "Allow",
            "Action": [
                "bedrock:InvokeModel",
                "bedrock:InvokeModelWithResponseStream"
            ],
            "Resource": [
                "arn:aws:bedrock:us-east-1::foundation-model/amazon.nova-pro-v1:0",
                "arn:aws:bedrock:us-east-2::foundation-model/amazon.nova-pro-v1:0",
                "arn:aws:bedrock:us-west-2::foundation-model/amazon.nova-pro-v1:0"
            ],
            "Condition": {
                "StringLike": {
                    "bedrock:InferenceProfileArn": "arn:aws:bedrock:us-west-2:794038231401:inference-profile/us.amazon.nova-pro-v1:0"
                }
            }
        }
    ]
}
```

In [1]:
import boto3
import json

In [2]:
session = boto3.Session()
sts_client = session.client('sts')
identity = sts_client.get_caller_identity()
print(identity['Arn'])

arn:aws:sts::794038231401:assumed-role/SageMaker-ExecutionRole-20250401T103257/SageMaker


In [3]:
bedrock = boto3.client(service_name='bedrock', region_name="us-west-2")
bedrock_runtime = boto3.client(service_name='bedrock-runtime', region_name="us-west-2")

In [4]:
inference_profile = "us.amazon.nova-pro-v1:0"

# If you'd like to try your own prompt, edit this parameter!
prompt_data = """Command: Write me a blog about making strong business decisions as a leader.

Blog:
"""

# Define one or more messages using the "user" and "assistant" roles.
message_list = [{"role": "user", "content": [{"text": prompt_data}]}]

# Configure the inference parameters.
inf_params = {"max_new_tokens": 250, "top_p": 0.9, "top_k": 20, "temperature": 0.7}

body = {
    "schemaVersion": "messages-v1",
    "messages": message_list,
    "inferenceConfig": inf_params,
}

In [5]:
# Attempt to invoke the model directly (may fail without inference profile)
try:
    response = bedrock_runtime.invoke_model(
        modelId=inference_profile,
        body=json.dumps(body),
        accept="application/json",
        contentType="application/json"
    )
    response_body = json.loads(response['body'].read().decode('utf-8'))
    print(response_body)
except Exception as e:
    print(f"Error: {e}")

{'output': {'message': {'content': [{'text': "### The Art of Making Strong Business Decisions as a Leader\n\nIn the dynamic world of business, the ability to make strong, informed decisions is a hallmark of effective leadership. Whether you're steering a startup or managing a multinational corporation, the decisions you make can significantly impact your organization's trajectory. Here’s a comprehensive guide to help you navigate the complexities of decision-making as a leader.\n\n#### 1. **Understanding the Decision-Making Process**\n\nStrong business decisions don’t happen in a vacuum. They are the result of a well-thought-out process that involves several key steps:\n\n**a. Identifying the Problem:**\n   - Clearly define the issue at hand. What are the symptoms, and what is the root cause? \n   - Gather data and input from various stakeholders to get a comprehensive view.\n\n**b. Gathering Information:**\n   - Collect relevant data and insights. This may include market research, fin

### Cross-Region Inference Requirement
The Amazon Nova Pro model may not be directly accessible in us-west-2 without cross-region inference. To address this, an inference profile is used to route requests to us-east-1, us-east-2, and us-west-2.